In [ ]:
# =============================================================================
# 0. Environment Setup
# =============================================================================
# !pip install transformers[torch] datasets accelerate sentencepiece scikit-learn matplotlib bitsandbytes -U

import json
import os
import torch
import random
import gc
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    get_scheduler,
    BitsAndBytesConfig
)
from sklearn.model_selection import train_test_split
from google.colab import drive

# =============================================================================
# 1. Hardware & Path Configuration
# =============================================================================
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/"

def set_seed(seed=42):
    """Ensure experimental reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# Automated BF16 detection for A100 optimization
if torch.cuda.is_available() and torch.cuda.is_bf16_supported():
    print("🚀 A100 detected: Enabling BF16 Mixed Precision for training.")
    dtype_config = torch.bfloat16
else:
    print("⚠️ BF16 not supported: Falling back to FP32.")
    dtype_config = torch.float32

CONFIG = {
    "deberta_model": "microsoft/deberta-v3-large",
    "llm_model": "Qwen/Qwen2.5-14B-Instruct",

    # Dataset Paths
    "train_files": [
        os.path.join(BASE_PATH, "dev_track_a.jsonl"),
        os.path.join(BASE_PATH, "synthetic_data_for_contrastive_learning.jsonl")
    ],
    "test_file":  os.path.join(BASE_PATH, "test_track_a.jsonl"),

    # Output Paths
    "output_file": os.path.join(BASE_PATH, "track_a_final_submission.jsonl"),
    "best_deberta_path": os.path.join(BASE_PATH, "best_deberta_fixed.pt"),
    "error_log_path": os.path.join(BASE_PATH, "error_analysis.json"),

    # Training Hyperparameters
    "max_len": 512,
    "batch_size": 8,
    "accum_steps": 4, # Effective batch size = 32
    "epochs": 4,
    "lr": 1e-5,
    "device": "cuda"
}

# =============================================================================
# 2. Data Engineering
# =============================================================================
def normalize_data(item):
    """Harmonize different data formats into a unified ranking structure."""
    def is_valid(s): return isinstance(s, str) and len(s.strip()) > 0

    # Format A: Standard Ranking
    if is_valid(item.get('anchor_text')) and is_valid(item.get('text_a')) and is_valid(item.get('text_b')):
        return item

    # Format B: Triplet Story (Contrastive Learning)
    if is_valid(item.get('anchor_story')) and is_valid(item.get('similar_story')) and is_valid(item.get('dissimilar_story